# Basic Text Mining and Document Classification
(by Tevfik Aytekin)

In [28]:
import nltk
from nltk.corpus import gutenberg
from nltk.probability import FreqDist
# You need to call nltk.download() to download all the nltk corpora

In [29]:
gutenberg.fileids()

['austen-emma.txt',
 'austen-persuasion.txt',
 'austen-sense.txt',
 'bible-kjv.txt',
 'blake-poems.txt',
 'bryant-stories.txt',
 'burgess-busterbrown.txt',
 'carroll-alice.txt',
 'chesterton-ball.txt',
 'chesterton-brown.txt',
 'chesterton-thursday.txt',
 'edgeworth-parents.txt',
 'melville-moby_dick.txt',
 'milton-paradise.txt',
 'shakespeare-caesar.txt',
 'shakespeare-hamlet.txt',
 'shakespeare-macbeth.txt',
 'whitman-leaves.txt']

In [3]:
gutenberg.raw('austen-emma.txt')

'[Emma by Jane Austen 1816]\n\nVOLUME I\n\nCHAPTER I\n\n\nEmma Woodhouse, handsome, clever, and rich, with a comfortable home\nand happy disposition, seemed to unite some of the best blessings\nof existence; and had lived nearly twenty-one years in the world\nwith very little to distress or vex her.\n\nShe was the youngest of the two daughters of a most affectionate,\nindulgent father; and had, in consequence of her sister\'s marriage,\nbeen mistress of his house from a very early period.  Her mother\nhad died too long ago for her to have more than an indistinct\nremembrance of her caresses; and her place had been supplied\nby an excellent woman as governess, who had fallen little short\nof a mother in affection.\n\nSixteen years had Miss Taylor been in Mr. Woodhouse\'s family,\nless as a governess than a friend, very fond of both daughters,\nbut particularly of Emma.  Between _them_ it was more the intimacy\nof sisters.  Even before Miss Taylor had ceased to hold the nominal\noffice o

In [30]:
print("num_chars: ", len(gutenberg.raw('austen-emma.txt')))
print("num_words: ", len(gutenberg.words('austen-emma.txt')))
print("num_sents: ", len(gutenberg.sents('austen-emma.txt')))

num_chars:  887071
num_words:  192427
num_sents:  7752


### Frequency Distribution

In [31]:
words = gutenberg.words('austen-emma.txt')
dist = FreqDist(words)

In [32]:
dist

FreqDist({',': 11454, '.': 6928, 'to': 5183, 'the': 4844, 'and': 4672, 'of': 4279, 'I': 3178, 'a': 3004, 'was': 2385, 'her': 2381, ...})

In [33]:
freqwords = [w for w in dist.keys() if len(w) > 5 and dist[w] > 50]
freqwords[:5]

['CHAPTER', 'Woodhouse', 'seemed', 'little', 'father']

### Stemming

The stemming process is defined in the [Porter stemmer web page](https://tartarus.org/martin/PorterStemmer/) as follows:

The Porter stemming algorithm (or ‘Porter stemmer’) is a process for removing the commoner morphological and inflexional endings from words in English. Its main use is as part of a term normalisation process that is usually done when setting up Information Retrieval systems.

In [34]:
text1 = "consulting consultant consultants consultings"
words1 = text1.lower().split(' ')
words1

['consulting', 'consultant', 'consultants', 'consultings']

In [35]:
porter = nltk.PorterStemmer()
[porter.stem(t) for t in words1]

['consult', 'consult', 'consult', 'consult']

In [36]:
text2 = "universal University universe"
words2 = text2.lower().split(' ')
[porter.stem(t) for t in words2]

['univers', 'univers', 'univers']

Note that the stemming process not always returns a whole meaningful word. This is because the aim of stemming is to increase the effectiveness of an information retrieval system and the success of a stemmer should be evaluated with respect to the effectiveness of the IR systems. If you want to get meaningful words then you should use lemmatization.

### Lemmatization

In [37]:
wn_lemma = nltk.WordNetLemmatizer()
text = "plays caring bats"
words = text.lower().split(' ')
[wn_lemma.lemmatize(t) for t in words]

['play', 'caring', 'bat']

In [38]:
wn_lemma.lemmatize('caring','v')

'care'

In [39]:
[wn_lemma.lemmatize(t) for t in words2]

['universal', 'university', 'universe']

### Part-of-speech (POS) Tagging

In [14]:
text = "Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence"
print(text)
words = nltk.word_tokenize(text)
nltk.pos_tag(words)

Emma Woodhouse, handsome, clever, and rich, with a comfortable home and happy disposition, seemed to unite some of the best blessings of existence


[('Emma', 'NNP'),
 ('Woodhouse', 'NNP'),
 (',', ','),
 ('handsome', 'NN'),
 (',', ','),
 ('clever', 'NN'),
 (',', ','),
 ('and', 'CC'),
 ('rich', 'JJ'),
 (',', ','),
 ('with', 'IN'),
 ('a', 'DT'),
 ('comfortable', 'JJ'),
 ('home', 'NN'),
 ('and', 'CC'),
 ('happy', 'JJ'),
 ('disposition', 'NN'),
 (',', ','),
 ('seemed', 'VBD'),
 ('to', 'TO'),
 ('unite', 'VB'),
 ('some', 'DT'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('best', 'JJS'),
 ('blessings', 'NNS'),
 ('of', 'IN'),
 ('existence', 'NN')]

## Document Classification

In [47]:
from nltk.corpus import stopwords, reuters
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier
stop_words = stopwords.words("english")
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
import pickle

In [48]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv("../datasets/bbc-text.csv")
# you can download the dataset from here: https://storage.googleapis.com/dataset-uploader/bbc/bbc-text.csv

In [49]:
data.head(10)

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
5,politics,howard hits back at mongrel jibe michael howar...
6,politics,blair prepares to name poll date tony blair is...
7,sport,henman hopes ended in dubai third seed tim hen...
8,sport,wilkinson fit to face edinburgh england captai...
9,entertainment,last star wars not for children the sixth an...


In [43]:
data['category'].value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64

In [50]:
data.iloc[0,1]

'tv future in the hands of viewers with home theatre systems  plasma high-definition tvs  and digital video recorders moving into the living room  the way people watch tv will be radically different in five years  time.  that is according to an expert panel which gathered at the annual consumer electronics show in las vegas to discuss how these new technologies will impact one of our favourite pastimes. with the us leading the trend  programmes and other content will be delivered to viewers via home networks  through cable  satellite  telecoms companies  and broadband service providers to front rooms and portable devices.  one of the most talked-about technologies of ces has been digital and personal video recorders (dvr and pvr). these set-top boxes  like the us s tivo and the uk s sky+ system  allow people to record  store  play  pause and forward wind tv programmes when they want.  essentially  the technology allows for much more personalised tv. they are also being built-in to high

### Vector space model

#### CountVectorizer


In [57]:
vectorizer = CountVectorizer(analyzer = "word", max_features = 10, max_df=0.3)
count_model = vectorizer.fit(data["text"])
X = count_model.transform(data["text"])

In [58]:
count_model.get_feature_names()

['against',
 'best',
 'do',
 'government',
 'only',
 'she',
 'them',
 'uk',
 'what',
 'you']

In [59]:
X.todense()[:5]

matrix([[0, 0, 1, 0, 1, 0, 5, 3, 7, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [4, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [56]:
X.shape

(2225, 10)

#### TfidfVectorizer


In [66]:
vectorizer = TfidfVectorizer(analyzer = "word", max_features = 10, max_df=0.3)
tfidf_model = vectorizer.fit(data["text"])
X = tfidf_model.transform(data["text"])

In [67]:
tfidf_model.get_feature_names()

['against',
 'best',
 'do',
 'government',
 'only',
 'she',
 'them',
 'uk',
 'what',
 'you']

In [68]:
X.todense()[:5]

matrix([[0.        , 0.        , 0.10935097, 0.        , 0.10411164,
         0.        , 0.54321921, 0.34228135, 0.74244945, 0.11716792],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 1.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.97278456, 0.        , 0.        , 0.        , 0.23171148,
         0.        , 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        , 0.        , 0.        , 0.        ]])

### Full model

In [69]:
vectorizer = TfidfVectorizer(analyzer = "word", max_features = 1000)
tfidf_model = vectorizer.fit(data["text"])
pickle.dump(tfidf_model, open("../datasets/tfidf.pkl", "wb"))
X = tfidf_model.transform(data["text"])
X_train,X_test,y_train,y_test = train_test_split(X,data["category"],test_size = 0.1)
clf = OneVsRestClassifier(LogisticRegression())
clf.fit(X_train, y_train)
pickle.dump(clf, open("../datasets/text_clf.pkl", 'wb'))
preds = clf.predict(X_test)
print(classification_report(y_test, preds))
print(confusion_matrix(y_test, preds))

 


               precision    recall  f1-score   support

     business       0.95      1.00      0.97        56
entertainment       0.91      0.94      0.93        34
     politics       1.00      0.91      0.95        45
        sport       0.98      0.98      0.98        51
         tech       0.95      0.95      0.95        37

     accuracy                           0.96       223
    macro avg       0.96      0.96      0.96       223
 weighted avg       0.96      0.96      0.96       223

[[56  0  0  0  0]
 [ 1 32  0  0  1]
 [ 2  1 41  0  1]
 [ 0  1  0 50  0]
 [ 0  1  0  1 35]]


In [72]:
tf_model = pickle.load(open("../datasets/tfidf.pkl", 'rb'))

new_data = ["The Betis substitute broke clear and coolly slide \
            the ball past goalkeeper Thibaut Courtois as the hosts \
            picked up only their eighth victory of the season to move \
            up to 12th, eight points clear of the relegation zone."]
X = tfidf_model.transform(new_data)
loaded_clf = pickle.load(open("../datasets/text_clf.pkl", 'rb'))
preds = loaded_clf.predict_proba(X)
print(loaded_clf.classes_)
print(preds)

['business' 'entertainment' 'politics' 'sport' 'tech']
[[0.20058321 0.09199421 0.10087989 0.5236559  0.08288679]]
